For the final project, I'll analyze the CoffeeKing dataset (Yelp Academic Dataset), to see if there's anything interesting we can dig out.  
Here's the discription:  
CoffeeKing is a new startup coffee company providing a unique and novel experience to their customers.  They want to appeal to a wide variety of clientele.  You will use Yelp reviews and user data to provide insights to CoffeeKing for things such as location selection and/or hours of operation (and other questions?)  
  
### First, import the packages we needed to do EDA.

In [ ]:
import pandas as pd
import numpy as np